In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.stats
import csv
import math

In [ ]:
with open("implausibilities", "r") as f:
    sb_implausibilities = pd.read_csv(f, index_col=0)
f.close()

with open("norm_dists", "r") as f:
    norm_dists = pd.read_csv(f, index_col=0)
f.close()

with open("model_variants", "r") as f:
    param_df = pd.read_csv(f, index_col=0)
f.close()

with open("outliers", "r") as f:
    obs_df2 = pd.read_csv(f, index_col=0)
f.close()

param_short_names = dict(zip(
    ['acure_prim_so4_diam', 'acure_sea_spray', 'acure_bvoc_soa', 'acure_dry_dep_acc'],
    ['Diam. primary anthro. SO4', 'Sea spray emission flux', 'BVOC SOA', 'Accumulation dry deposition rate']
))

sb_cv = np.sqrt(scipy.stats.chi2.ppf(0.95, sum((~obs_df2.missing) & (~obs_df2.outlier))))

In [ ]:
def show_parametric_constraint(method='sb', q=0.95, save=False):
    
    my_param_df = param_df.copy()
    
    if method=='sb':
        my_param_df['implausibilities'] = sb_implausibilities
        title = 'Strict bounds implausibilities'
        cv = sb_cv
    elif method=='hm':
        my_param_df['implausibilities'] = np.nanquantile(norm_dists, q=q, axis=0)
        title = 'History matching implausibilities'
        cv = np.quantile([np.quantile(scipy.stats.halfnorm.rvs(size=sum((~obs_df2.missing) & (~obs_df2.outlier))), q=q) for i in range(5000)], q=0.95)

    my_param_df['colors'] = (my_param_df['implausibilities']>cv)
    ylim = [
        min(math.floor(cv), math.floor(np.min(my_param_df['implausibilities']))), 
        max(math.ceil(cv), math.ceil(np.max(my_param_df['implausibilities'])))
    ]

    fig, (ax0, ax1, ax2) = plt.subplots(
        nrows=3,
        ncols=1,
        sharey=False,
        figsize=(2, 4),
        dpi=1200
    )
    fig.tight_layout()

    ax0.set_xlabel(param_short_names['acure_sea_spray'], fontsize=8)
    ax0.set_xlim(0.25, 4)
    ax0.set_xticks([0.25, 4])
    ax0.set_ylim(*ylim)
    ax0.set_yticks(ylim)
    ax0.tick_params(axis='both', which='major', labelsize=8)
    ax0.scatter(
        my_param_df['acure_sea_spray'],
        my_param_df['implausibilities'],
        alpha=1,
        s=0.01,
        c=my_param_df['colors'],
        cmap='bwr'
    )
    ax0.axhline(
        cv,
        c='r'
    )

    ax1.set_xlabel(param_short_names['acure_bvoc_soa'], fontsize=8)
    ax1.set_xlim(0.32, 3.68)
    ax1.set_xticks([0.32, 3.68])
    ax1.set_ylim(*ylim)
    ax1.set_yticks(ylim)
    ax1.tick_params(axis='both', which='major', labelsize=8)
    ax1.scatter(
        my_param_df['acure_bvoc_soa'],
        my_param_df['implausibilities'],
        alpha=1,
        s=0.01,
        c=my_param_df['colors'],
        cmap='bwr'
    )
    ax1.axhline(
        cv,
        c='r'
    )

    ax2.set_xlabel(param_short_names['acure_dry_dep_acc'], fontsize=8)
    ax2.set_xlim(0.1, 10)
    ax2.set_xticks([0.1, 10])
    ax2.set_ylim(*ylim)
    ax2.set_yticks(ylim)
    ax2.tick_params(axis='both', which='major', labelsize=8)
    ax2.scatter(
        my_param_df['acure_dry_dep_acc'],
        my_param_df['implausibilities'],
        alpha=1,
        s=0.01,
        c=my_param_df['colors'],
        cmap='bwr'
    )
    ax2.axhline(
        cv,
        c='r'
    )

    fig.text(0, 0.5, title, va='center', rotation='vertical', fontsize=8)
    if method=='hm':
        fig.suptitle('Quantile ' + str(q), fontsize=8, y=1.02)

    plt.show()

In [ ]:
def show_biparametric_constraints(pars=list(param_short_names.keys()), method='sb', q=0.95, save=False):
    """
    Arguments
    
    method : 
    """
    
    pairs = [[a, b] for idx, a in enumerate(pars) for b in pars[idx + 1:]]
    
    for pair in pairs:
        binned_param_df = pd.DataFrame()
        my_param_df = param_df.copy()

        binned_param_df[pair[0]], bins_0 = pd.cut(my_param_df[pair[0]], 15, labels=list(range(15)), retbins=True)
        binned_param_df[pair[1]], bins_1 = pd.cut(my_param_df[pair[1]], 15, labels=list(range(15)), retbins=True)
        if method=='sb':
            plausible = (sb_implausibilities <= sb_cv)
            binned_param_df['plausible'] = plausible.reset_index(drop=True)
            binned_param_df['implausibilities'] = sb_implausibilities.reset_index(drop=True)
        elif method=='hm':
            hm_implausibilities = np.nanquantile(norm_dists, q=q, axis=0)
            hm_cv = np.quantile([np.quantile(scipy.stats.halfnorm.rvs(size=sum((~obs_df2.missing) & (~obs_df2.outlier))), q=q) for i in range(5000)], q=0.95)
            hm_plausible = (hm_implausibilities <= hm_cv)
            binned_param_df['plausible'] = hm_plausible
            binned_param_df['implausibilities'] = hm_implausibilities

        binned_param_df = binned_param_df.replace({
            pair[0]: dict(zip(list(range(15)), bins_0)),
            pair[1]: dict(zip(list(range(15)), bins_1))
        })

        grouped = binned_param_df.groupby(pair).agg({'plausible': 'mean'}).reset_index()
        grouped_implausible = grouped[grouped.plausible==0]

        fig = plt.figure(figsize=(5.67, 5), dpi=1200)

        plt.scatter(grouped_implausible[pair[0]], grouped_implausible[pair[1]], c=grouped_implausible.plausible, marker='s', s=110, alpha=1)
        plt.scatter(grouped[pair[0]], grouped[pair[1]], c=grouped.plausible, marker='s', s=110, alpha=0.5)
        plt.xlabel(param_short_names[pair[0]], fontsize=8)
        plt.ylabel(param_short_names[pair[1]], fontsize=8)
        cbar = plt.colorbar()
        cbar.ax.tick_params(labelsize=8)
        cbar.set_label(label='Proportion plausible', fontsize=8)

        if save:
            plt.savefig(method + '_constraint_' + pair[0] + '_' + pair[1])
        plt.show()

# Strict bounds

In [ ]:
show_parametric_constraint(method='sb', save=False)

In [ ]:
show_biparametric_constraints(method='sb')

# History matching

In [ ]:
q_quantiles = [np.quantile(scipy.stats.halfnorm.rvs(size=100), q=0.95) for i in range(10000)]
np.quantile(q_quantiles, q=0.95)

In [ ]:
for q in [0.75, 0.9, 0.95]:
    show_parametric_constraint(method='hm', q=q)

In [ ]:
for q in [0.75, 0.9, 0.95]:
    show_biparametric_constraints(method='hm', q=q)